LAVAのスクレイピング

In [14]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np

1件目の情報を取得する。

In [15]:
base_url = "https://yoga-lava.com/re_search/studio.html?code=7" 
response = requests.get(base_url)
soup     = BeautifulSoup(response.content, "lxml")
items    = soup.find(class_="shopList")

In [16]:
# スクレイピングした HTML情報出力,取得したデータをテキストファイルに保存するための手順
#f = open("test3.txt", "w", encoding="utf-8") #エラーが出たためencoding="utf-8"を追加
#f.write(str(items))
#f.close()

In [17]:
# 各情報の取得

# 店舗名を含む要素を検索
shop_name_element = items.find('a', href=re.compile(r'^/tokyo/[^/]+/$'))
# 店舗名を取得
shop_name = shop_name_element.get_text(strip=True) if shop_name_element else None

#住所
shop_address    = items.find('p').get_text() if items.find('p') else None

#店舗情報リンク
# <a>タグのhref属性を取得
link_elements = items.find('a', href=re.compile(r'^/tokyo/[^/]+/$'))
# 各リンクを完全なURLに変換
base_url = "https://yoga-lava.com"
complete_url = base_url + link_elements['href']


print("店舗名 (Name):", shop_name)
print("住所 (Address):", shop_address)
print("リンク (Link):",complete_url)

店舗名 (Name): ホットヨガスタジオLAVA秋葉原店
住所 (Address): 〒101-0023東京都千代田区神田松永町1　宮沢ビル4F
リンク (Link): https://yoga-lava.com/tokyo/akihabara/


東京都全店舗の情報を取得する。

In [18]:
# 基本URLの設定
url = "https://yoga-lava.com/re_search/studio.html?code=7"

all_data = []

response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')
items = soup.findAll(class_="shopList")

print("items", len(items))

for item in items:
    base_data = {}

    tbodys = item.findAll("dl")

    for tbody in tbodys:
        data = base_data.copy()

        shop_name_element = tbody.select_one('a', href=re.compile(r'^/tokyo/[^/]+/$'))
        data["店舗名"]  = shop_name_element.get_text(strip=True) if shop_name_element else None

        data["住所"]    = tbody.select_one('p').get_text(strip=True) if tbody.select_one('p') else None


        link_elements = tbody.select_one('a', href=re.compile(r'^/tokyo/[^/]+/$'))
        base_url = "https://yoga-lava.com"
        data["リンク"]  = base_url + link_elements['href']

        all_data.append(data)

# 結果を表示
for data in all_data:
    print(data)


items 40
{'店舗名': 'ホットヨガスタジオLAVA秋葉原店', '住所': '〒101-0023東京都千代田区神田松永町1\u3000宮沢ビル4F', 'リンク': 'https://yoga-lava.com/tokyo/akihabara/'}
{'店舗名': 'ホットヨガ＆Evolv EMSトレーニングLAVA銀座本店', '住所': '〒104-0061東京都中央区銀座3-3-1\u3000ZOE銀座4F', 'リンク': 'https://yoga-lava.com/tokyo/ginza/'}
{'店舗名': 'ホットヨガ＆マシンピラティスLAVA銀座グラッセ店', '住所': '〒104-0061東京都中央区銀座3-2-15\u3000ギンザ・グラッセ6F', 'リンク': 'https://yoga-lava.com/tokyo/ginza-glasse/'}
{'店舗名': 'ホットヨガスタジオLAVA月島店', '住所': '〒104-0052東京都中央区月島1-8-3\u3000BR月島1ビル3F', 'リンク': 'https://yoga-lava.com/tokyo/tsukishima/'}
{'店舗名': 'ホットヨガスタジオLAVA赤坂店', '住所': '〒107-0052東京都港区赤坂4-3-6\u3000赤坂一ツ木通りビル6F', 'リンク': 'https://yoga-lava.com/tokyo/akasaka/'}
{'店舗名': 'ホットヨガ＆ナイトサーフィンLAVA新宿東口店', '住所': '〒160-0022東京都新宿区新宿3-36-6\u3000新宿大安ビル5F', 'リンク': 'https://yoga-lava.com/tokyo/shinjyukuhigashi/'}
{'店舗名': 'ホットヨガスタジオLAVA高田馬場店', '住所': '〒169-0075東京都新宿区高田馬場2-16-6\u3000宇田川ビル3F', 'リンク': 'https://yoga-lava.com/tokyo/takadanobaba/'}
{'店舗名': 'ホットヨガスタジオLAVA市ヶ谷店', '住所': '〒162-0844東京都新宿区市谷八幡町13\u3000東京洋服会館5F', 'リンク': 'h

In [19]:
df = pd.DataFrame(all_data)
df.head()

,店舗名,住所,リンク
0,ホットヨガスタジオLAVA秋葉原店,〒101-0023東京都千代田区神田松永町1 宮沢ビル4F,https://yoga-lava.com/tokyo/akihabara/
1,ホットヨガ＆Evolv EMSトレーニングLAVA銀座本店,〒104-0061東京都中央区銀座3-3-1 ZOE銀座4F,https://yoga-lava.com/tokyo/ginza/
2,ホットヨガ＆マシンピラティスLAVA銀座グラッセ店,〒104-0061東京都中央区銀座3-2-15 ギンザ・グラッセ6F,https://yoga-lava.com/tokyo/ginza-glasse/
3,ホットヨガスタジオLAVA月島店,〒104-0052東京都中央区月島1-8-3 BR月島1ビル3F,https://yoga-lava.com/tokyo/tsukishima/
4,ホットヨガスタジオLAVA赤坂店,〒107-0052東京都港区赤坂4-3-6 赤坂一ツ木通りビル6F,https://yoga-lava.com/tokyo/akasaka/


住所から郵便番号を削除

In [20]:
# 郵便番号を削除する関数
def remove_postal_code(address):
    return re.sub(r'〒\d{3}[-‐]\d{4}', '', address).strip()

# 住所から郵便番号を削除
df['住所'] = df['住所'].apply(remove_postal_code)

# 結果の表示
df.head()

,店舗名,住所,リンク
0,ホットヨガスタジオLAVA秋葉原店,東京都千代田区神田松永町1 宮沢ビル4F,https://yoga-lava.com/tokyo/akihabara/
1,ホットヨガ＆Evolv EMSトレーニングLAVA銀座本店,東京都中央区銀座3-3-1 ZOE銀座4F,https://yoga-lava.com/tokyo/ginza/
2,ホットヨガ＆マシンピラティスLAVA銀座グラッセ店,東京都中央区銀座3-2-15 ギンザ・グラッセ6F,https://yoga-lava.com/tokyo/ginza-glasse/
3,ホットヨガスタジオLAVA月島店,東京都中央区月島1-8-3 BR月島1ビル3F,https://yoga-lava.com/tokyo/tsukishima/
4,ホットヨガスタジオLAVA赤坂店,東京都港区赤坂4-3-6 赤坂一ツ木通りビル6F,https://yoga-lava.com/tokyo/akasaka/


国土地理院APIで緯度経度を取得

In [21]:
import requests
# APIのURL
url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

# 住所から緯度経度を取得する関数
def get_lat_lon(address):
    try:
        res = requests.get(url + address)
        res.raise_for_status()  # HTTPエラーが発生した場合に例外を発生させる
        data = res.json()
        if data and len(data) > 0:
            location = data[0]['geometry']['coordinates']
            return location[1], location[0]  # 緯度、経度を返す
        else:
            return None, None
    except Exception as e:
        print(f"Error retrieving location for address {address}: {e}")
        return None, None

# 緯度経度の取得とデータフレームへの保存
df[['緯度', '経度']] = df['住所'].apply(lambda x: pd.Series(get_lat_lon(x)))

# 結果の表示
df.head()

,店舗名,住所,リンク,緯度,経度
0,ホットヨガスタジオLAVA秋葉原店,東京都千代田区神田松永町1 宮沢ビル4F,https://yoga-lava.com/tokyo/akihabara/,35.699150,139.775208
1,ホットヨガ＆Evolv EMSトレーニングLAVA銀座本店,東京都中央区銀座3-3-1 ZOE銀座4F,https://yoga-lava.com/tokyo/ginza/,35.673626,139.765762
2,ホットヨガ＆マシンピラティスLAVA銀座グラッセ店,東京都中央区銀座3-2-15 ギンザ・グラッセ6F,https://yoga-lava.com/tokyo/ginza-glasse/,35.673225,139.764465
3,ホットヨガスタジオLAVA月島店,東京都中央区月島1-8-3 BR月島1ビル3F,https://yoga-lava.com/tokyo/tsukishima/,35.664688,139.783707
4,ホットヨガスタジオLAVA赤坂店,東京都港区赤坂4-3-6 赤坂一ツ木通りビル6F,https://yoga-lava.com/tokyo/akasaka/,35.674438,139.735504


In [22]:
# 住所から「〇〇区」を抽出する関数
def extract_ku(address):
    match = re.search(r'東京都(\w+区)', address)
    return match.group(1) if match else None

# 新しいカラム「区」を作成
df['区'] = df['住所'].apply(extract_ku)

# 結果の表示
df.head()

,店舗名,住所,リンク,緯度,経度,区
0,ホットヨガスタジオLAVA秋葉原店,東京都千代田区神田松永町1 宮沢ビル4F,https://yoga-lava.com/tokyo/akihabara/,35.699150,139.775208,千代田区
1,ホットヨガ＆Evolv EMSトレーニングLAVA銀座本店,東京都中央区銀座3-3-1 ZOE銀座4F,https://yoga-lava.com/tokyo/ginza/,35.673626,139.765762,中央区
2,ホットヨガ＆マシンピラティスLAVA銀座グラッセ店,東京都中央区銀座3-2-15 ギンザ・グラッセ6F,https://yoga-lava.com/tokyo/ginza-glasse/,35.673225,139.764465,中央区
3,ホットヨガスタジオLAVA月島店,東京都中央区月島1-8-3 BR月島1ビル3F,https://yoga-lava.com/tokyo/tsukishima/,35.664688,139.783707,中央区
4,ホットヨガスタジオLAVA赤坂店,東京都港区赤坂4-3-6 赤坂一ツ木通りビル6F,https://yoga-lava.com/tokyo/akasaka/,35.674438,139.735504,港区


データベースの作成

In [23]:
import sqlite3

In [24]:
# データベース接続を作成
conn = sqlite3.connect('property.db')

In [25]:
# データフレームをSQLiteテーブルに変換
df.to_sql('LAVA', conn, if_exists='replace', index=False)

100

In [26]:
conn.close()